# Rosette phenotyping

use conda activate PhenoLeaf to set the container environment active

## Image corrections

### Image optical-distortion correction

Images of full tray displays some defects whixh may be linked to optical distortion.
It is observed a barrel type of radial distortion i.e. plants at the center of the tray tends to be bigger than they are and conversly plants at the edges of the tray tends to be small. In addition these size defects that can be easyly observed on the Fig 0. Optical distortions also tiggers modifications in shapes and sizes of image's pixels. The causes of image optical-distortion can be various (due to inegal lens magnification, camera sensor, non parrallel plane-object).

![texte alternatif](/home/mcaroulle/Pheno_Leaf/Image optical-distortion.png "titre")

Thus it is necessary to fix these kinds of optical defects. We use an open-source python package named "Discorpy" wich github repository can be found here https://github.com/algotom/discorpy/tree/master

This package use a 3 step process:
- pre-processing : extracting and grouping reference-points from a calibration image
- processing : caculation of coefficients of correction for the model
- post-processing : correcting images


We used to calibrate the commercial camera used (Canon EOS800D) using a chessboard image.
The chessboard pattern was generated using following website:
https://markhedleyjones.com/projects/calibration-checkerboard-collection
(4 A4 chesscheckboards were stuck together and used a a single calibration checkboard).




In [1]:
import numpy as np
import discorpy
import discorpy.losa.loadersaver as io
import discorpy.prep.preprocessing as prep
import discorpy.prep.linepattern as lprep
import discorpy.proc.processing as proc
import discorpy.post.postprocessing as post

# Initial parameters
file_path = "/home/mcaroulle/Pheno_Leaf/PL_Final/RAW_INPUT/Real-chessboard.JPG" #import the chessboradcheck image
output_base = "/home/mcaroulle/Pheno_Leaf/PL_Final/RAW_OUTPUT"
num_coef = 5  # Number of polynomial coefficients
mat0 = io.load_image(file_path) # Load image
(height, width) = mat0.shape

# Convert the chessboard image to a line-pattern image
mat1 = lprep.convert_chessboard_to_linepattern(mat0)
io.save_image(output_base + "/line_pattern_converted.jpg", mat1)

# Calculate slope and distance between lines
slope_hor, dist_hor = lprep.calc_slope_distance_hor_lines(mat1, radius=15, sensitive=0.5)
slope_ver, dist_ver = lprep.calc_slope_distance_ver_lines(mat1, radius=15, sensitive=0.5)
print("Horizontal slope: ", slope_hor, " Distance: ", dist_hor)
print("Vertical slope: ", slope_ver, " Distance: ", dist_ver)

Horizontal slope:  -0.002617999859155802  Distance:  200.35972055719603
Vertical slope:  -0.005236035605700117  Distance:  199.7484348376889


In [2]:
# Extract reference-points
list_points_hor_lines = lprep.get_cross_points_hor_lines(mat1, slope_ver, dist_ver,
                                                         ratio=0.3, norm=True, offset=450,
                                                         bgr="bright", radius=15,
                                                         sensitive=0.5, denoise=True,
                                                         subpixel=True)
list_points_ver_lines = lprep.get_cross_points_ver_lines(mat1, slope_hor, dist_hor,
                                                         ratio=0.3, norm=True, offset=150,
                                                         bgr="bright", radius=15,
                                                         sensitive=0.5, denoise=True,
                                                         subpixel=True)
if len(list_points_hor_lines) == 0 or len(list_points_ver_lines) == 0:
    raise ValueError("No reference-points detected !!! Please adjust parameters !!!")
io.save_plot_points(output_base + "/ref_points_horizontal.png", list_points_hor_lines,
                    height, width, color="red")
io.save_plot_points(output_base + "/ref_points_vertical.png", list_points_ver_lines,
                    height, width, color="blue")

# Group points into lines
list_hor_lines = prep.group_dots_hor_lines(list_points_hor_lines, slope_hor, dist_hor,
                                           ratio=0.1, num_dot_miss=2, accepted_ratio=0.8)
list_ver_lines = prep.group_dots_ver_lines(list_points_ver_lines, slope_ver, dist_ver,
                                           ratio=0.1, num_dot_miss=2, accepted_ratio=0.8)

# Remove residual dots
list_hor_lines = prep.remove_residual_dots_hor(list_hor_lines, slope_hor, 2.0)
list_ver_lines = prep.remove_residual_dots_ver(list_ver_lines, slope_ver, 2.0)

# Save output for checking
io.save_plot_image(output_base + "/horizontal_lines.png", list_hor_lines, height, width)
io.save_plot_image(output_base + "/vertical_lines.png", list_ver_lines, height, width)
list_hor_data = post.calc_residual_hor(list_hor_lines, 0.0, 0.0)
list_ver_data = post.calc_residual_ver(list_ver_lines, 0.0, 0.0)
io.save_residual_plot(output_base + "/hor_residual_before_correction.png",
                      list_hor_data, height, width)
io.save_residual_plot(output_base + "/ver_residual_before_correction.png",
                      list_ver_data, height, width)

'/home/mcaroulle/Pheno_Leaf/PL_Final/RAW_OUTPUT/ver_residual_before_correction.png'

In [3]:
# Regenerate grid points after correcting the perspective effect.
list_hor_lines, list_ver_lines = proc.regenerate_grid_points_parabola(
    list_hor_lines, list_ver_lines, perspective=True)

# Calculate parameters of the radial correction model
(xcenter, ycenter) = proc.find_cod_coarse(list_hor_lines, list_ver_lines)
list_fact = proc.calc_coef_backward(list_hor_lines, list_ver_lines,
                                    xcenter, ycenter, num_coef)
io.save_metadata_txt(output_base + "/coefficients_radial_distortion.txt",
                     xcenter, ycenter, list_fact)

print("X-center: {0}. Y-center: {1}".format(xcenter, ycenter))
print("Coefficients: {0}".format(list_fact))

# Check the correction results:
# Apply correction to the lines of points
list_uhor_lines = post.unwarp_line_backward(list_hor_lines, xcenter, ycenter,
                                            list_fact)
list_uver_lines = post.unwarp_line_backward(list_ver_lines, xcenter, ycenter,
                                            list_fact)
# Calculate the residual of the unwarpped points.
list_hor_data = post.calc_residual_hor(list_uhor_lines, xcenter, ycenter)
list_ver_data = post.calc_residual_ver(list_uver_lines, xcenter, ycenter)
# Save the results for checking
io.save_plot_image(output_base + "/unwarpped_horizontal_lines.png",
                   list_uhor_lines, height, width)
io.save_plot_image(output_base + "/unwarpped_vertical_lines.png",
                   list_uver_lines, height, width)
io.save_residual_plot(output_base + "/hor_residual_after_correction.png",
                      list_hor_data, height, width)
io.save_residual_plot(output_base + "/ver_residual_after_correction.png",
                      list_ver_data, height, width)

X-center: 2681.444742082234. Y-center: 1983.8462758586431
Coefficients: [ 1.00840054e+00 -1.03875450e-05 -6.37773587e-09  1.28491195e-12
 -1.52520141e-16]


'/home/mcaroulle/Pheno_Leaf/PL_Final/RAW_OUTPUT/ver_residual_after_correction.png'

After the coefficents corrections are calculated, we can apply them to images

In [ ]:

# Load coefficients from previous calculation
(xcenter, ycenter, list_fact) = io.load_metadata_txt(
    output_base + "/coefficients_radial_distortion.txt")

# Load an image and correct it.
img = io.load_image("/home/mcaroulle/Pheno_Leaf/test.JPG", average=False)
img_corrected = np.copy(img)
for i in range(img.shape[-1]):
    img_corrected[:, :, i] = post.unwarp_image_backward(img[:, :, i], xcenter,
                                                        ycenter, list_fact)
io.save_image(output_base + "/TRUE_image_corrected.jpg", img_corrected)

### Color normalization

To improve natural python thresholding commands it is better to homogenize the color of our images likewise the pipeline could be applied to all the dataset minimizing errors due to color differences and allowing to set the same thresholding parameters for all the images.


Our images displays a large variety of colors and texturess which can make more difficult this color uniformisation by simple color modifications.

Here we propose to use a pipeline designed in the following study to perform color normalisation using machine learning and a method described in https://www.cs.tau.ac.il/~turkel/imagepapers/ColorTransfer.pdf

In [ ]:
import numpy as np
import cv2
import os

#Set input directory
input_dir="/home/mcaroulle/Pheno_Leaf/Data/Color normalization/Input/"
input_image_list=os.listdir(input_dir)

#Set output directory
output_dir="/home/mcaroulle/Pheno_Leaf/Data/Color normalization/Output/"

def get_mean_and_std(x):
	x_mean, x_std = cv2.meanStdDev(x)
	x_mean = np.hstack(np.around(x_mean,2))
	x_std = np.hstack(np.around(x_std,2))
	return x_mean, x_std

#Set input for the template library
template_img = cv2.imread('/home/mcaroulle/Pheno_Leaf/Data/Color normalization/Template/1.png')
template_img = cv2.cvtColor(template_img,cv2.COLOR_BGR2LAB)
template_mean, template_std = get_mean_and_std(template_img)

for img in (input_image_list):
    print(img)
    input_img = cv2.imread(input_dir+img)
    input_img = cv2.cvtColor(input_img,cv2.COLOR_BGR2LAB)
    
    
    img_mean, img_std = get_mean_and_std(input_img)
    
    
    height, width, channel = input_img.shape
    for i in range(0,height):
        for j in range(0,width):
            for k in range(0,channel):
                x = input_img[i,j,k]
                x = ((x-img_mean[k])*(template_std[k]/img_std[k]))+template_mean[k]
                x = round(x)
                x = 0 if x<0 else x
                x = 255 if x>255 else x
                input_img[i,j,k] = x
            
    input_img= cv2.cvtColor(input_img,cv2.COLOR_LAB2BGR)
    cv2.imwrite(output_dir+"modified_"+img, input_img)

## Image Segmentation

### Noise Cleaning

In [ ]:
#Let's clean the noise using edge preserving filter.
#As mentioned in previous tutorial, my favorite is NLM

from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import img_as_ubyte, img_as_float

float_img = img_as_float(img)
sigma_est = np.mean(estimate_sigma(float_img, multichannel=True))


denoise_img = denoise_nl_means(float_img, h=1.15 * sigma_est, fast_mode=False, 
                               patch_size=5, patch_distance=3, multichannel=True)
                           
denoise_img_as_8byte = img_as_ubyte(denoise_img)
#plt.imshow(denoise_img_as_8byte, cmap=plt.cm.gray, interpolation='nearest')

### Color segmentation using histogram

In [ ]:
#Lybraries importation
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage import io, measure

#Image path
img=io.imread('/home/mcaroulle/Pheno_Leaf/Data/Color segmentation/13.png')
#plt.imshow(img)




##convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
mask= cv2.inRange(hsv,(30,20,150),(70,255,255))

#plt.imshow(mask)

from scipy import ndimage as nd
closed_mask = nd. binary_closing(mask, np. ones ((7,7)))
plt.imshow(closed_mask)

label_image= measure.label(closed_mask)
#plt.imshow(label_image)

from skimage.color import label2rgb
image_label_overlay = label2rgb(label_image, image= img)
#plt.imshow(image_label_overlay)

props = measure.regionprops_table(label_image, img, 
                                  properties=['label','area','equivalent_diameter'])

import pandas as pd

df = pd.DataFrame(props)
#print(df.head())

### Filter

In [ ]:
# Libraries importation
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage import io, measure

# Image path
img = io.imread('/home/mcaroulle/Pheno_Leaf/Data/Color segmentation/13.png')

## Convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
mask = cv2.inRange(hsv, (30, 20, 150), (70, 255, 255))

from scipy import ndimage as nd
closed_mask = nd.binary_closing(mask, np.ones((7, 7)))

label_image = measure.label(closed_mask)

props = measure.regionprops_table(label_image, img,
                                  properties=['label', 'area', 'centroid'])

# Convert props to DataFrame
df = pd.DataFrame(props)

# Filter regions with area less than 5,000
df_filtered = df[df['area'] >= 5000]
print(df_filtered)

# Filter label_image based on the filtered DataFrame
filtered_label_image = np.isin(label_image, df_filtered['label'])

# Show the filtered label_image
plt.imshow(filtered_label_image)
plt.show()

### Label fusion


In [ ]:
#function wich fuse all the labels contained in a list

def merge_labels(label_image, df):
    list_labels = df['label'].tolist()

    for k in range (len(list_labels)-1):
        merged_label_image = np.copy(label_image)
        merged_label_image[merged_label_image == list_labels[k+1]] = list_labels[k]

    return merged_label_image



merged_label_image = merge_labels(filtered_label_image, df_filtered)
plt.imshow(merged_label_image)



### Image skeletonization

In [ ]:
from skimage.morphology import skeletonize
from skimage import data
import matplotlib.pyplot as plt
from skimage.util import invert

# Invert the horse image
image = merged_label_image

# perform skeletonization
skeleton = skeletonize(image)

# display results
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4),
                         sharex=True, sharey=True)

ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].axis('off')
ax[0].set_title('original', fontsize=20)

ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].axis('off')
ax[1].set_title('skeleton', fontsize=20)

fig.tight_layout()
plt.show()

### Watershed segmentation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max


image = merged_label_image

# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(image)
coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=image)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, _ = ndi.label(mask)
labels = watershed(-distance, markers, mask=image)

fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

## Object detection

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cv2
import numpy as np
from skimage import data,io,measure
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from scipy import ndimage as nd

#Image path
image=io.imread('/home/mcaroulle/Pheno_Leaf/Data/Color segmentation/13.png')


##convert to hsv
hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
mask= cv2.inRange(hsv,(30,20,150),(70,255,255))

closed_mask = nd. binary_closing(mask, np. ones ((7,7)))


label_image= measure.label(closed_mask)

cleared = clear_border(label_image)

# label image regions
label_image = label(cleared)
# to make the background transparent, pass the value of `bg_label`,
# and leave `bg_color` as `None` and `kind` as `overlay`
image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for region in regionprops(label_image):
    # take regions with large enough areas
    if region.area >= 5000:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

## whole tray segmentation

### Segmentation

In [ ]:
#Lybraries importation
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage import io, measure
from scipy import ndimage as nd
from skimage.color import label2rgb
import pandas as pd


#Image path
img=io.imread('/home/mcaroulle/Pheno_Leaf/Data/Color segmentation/TRAY_1.png')
#plt.imshow(img)

##convert to hsv
hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
mask= cv2.inRange(hsv,(30,20,150),(70,255,255))

closed_mask = nd.binary_closing(mask, np. ones ((7,7)))


label_image= measure.label(closed_mask)



image_label_overlay = label2rgb(label_image, image= img)
plt.imshow(image_label_overlay)

props = measure.regionprops_table(label_image, img, 
                                  properties=['label','area','centroid','coords'])



df = pd.DataFrame(props)
#print(df.head())

### Filtering

#### Remove border artefacts 

In [ ]:
from skimage import segmentation

# remove artifacts connected to image border
cleared_label_image = segmentation.clear_border(label_image)

#take measurements 
props = measure.regionprops_table(cleared_label_image, img,
                                  properties=['label', 'area', 'centroid','coords'])

# Convert props to DataFrame
df = pd.DataFrame(props)

plt.imshow(cleared_label_image)


#### Filtering by area


In [ ]:
# Filter regions with area less than 5,000
df_filtered = df[df['area'] >= 500]


# Filter label_image based on the filtered DataFrame
filtered_label_image = np.isin(cleared_label_image , df_filtered['label'])

# Show the filtered label_image
plt.imshow(filtered_label_image)

#### Centroids plot

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd

# Charger l'image
image = filtered_label_image

# Créer la figure et les axes
fig, ax = plt.subplots()

# Afficher l'image
ax.imshow(image)

centrioles_x = df_filtered['centroid-0'].tolist()
centrioles_y = df_filtered['centroid-1'].tolist()

# Tracer les centrioles
for x, y in zip(centrioles_x, centrioles_y):
    circle = patches.Circle((x, y), radius=5, color='red')
    ax.add_patch(circle)

# Afficher le tracé
plt.show()


#### Tray Slicing

In [ ]:
from skimage.io import imread
import pandas as pd

# Charger l'image
image = imread('/home/mcaroulle/Pheno_Leaf/Data/Color segmentation/TRAY_1.png')

# Extraire les dimensions de l'image
height, width, channels = image.shape

#Calcul de la largeur et de la hauteur des pots dans le tray
x_pot = width/7
y_pot = height/5



# Création des données qui correspond à un découpage du tray en pots
x_min=[]
x_max=[]
X=[]

for k in range(5):
    x=0
    for k in range (8):
        X.append(int(x))
        x+=x_pot
    x_min+=X[0:7]
    x_max+=X[1:8]
    if x_max[-1]!=width:
        x_max[-1]=width


y_min=[]
y_max=[]
Y=[]


for k in range(7):
    y=0
    for k in range (6):
        Y.append(int(y))
        y+=y_pot
    y_min+=Y[0:5]
    y_max+=Y[1:6]
    if y_max[-1]!=height:
        y_max[-1]=height


data_tray_max = {'Pot_position': [i for i in range(1, 36)],
        'xmin': [k for k in x_min],
        'xmax': [k for k in x_max],
        'ymin': [k for k in y_min],
        'ymax': [k for k in y_max]}

df_tray_max = pd.DataFrame(data_tray_max)
#print(df_tray_max)


##Opération sur df_tray_max pour créer df_tray_min

#Calculation of the rejected zone
x_R_pot = int((width/7)/20)
y_R_pot = int((height/5)/20)


df_tray_min = pd.DataFrame(data_tray_max)


df_tray_min['xmin'] = df_tray_min['xmin'] + x_R_pot
df_tray_min['xmax'] = df_tray_min['xmax'] - x_R_pot
df_tray_min['ymin'] = df_tray_min['ymin'] + y_R_pot
df_tray_min['ymax'] = df_tray_min['ymax'] - y_R_pot

##création du tableau ne contenant que les centroïdes en filtrant le dataframe du masque obtenu précemment
df_centroid = df_filtered [['label','centroid-0','centroid-1']]



# BROUILLON


In [ ]:
# on crée un df vide pour recueillir les résultats
data_resultat = {'label': [],
        'centroid-0': [],
        'centroid-1': []}

df_resultat = pd.DataFrame(data_resultat)






import pandas as pd



for k in range (len(df_tray_max)):

    ##### CENTROIDES PRESENTS DANS TRAY MAX

    df_subset_centroid_max= df_centroid[(df_centroid['centroid-0'] >= df_tray_max['xmin'][k]) & (df_centroid['centroid-0'] <= df_tray_max['xmax'][k]) & (df_centroid['centroid-1'] >= df_tray_max['ymin'][k]) & (df_centroid['centroid-1'] <= df_tray_max['ymax'][k])]


    #### CENTROIDES PRESENTS DANS LE TRAY MIN

    # Filtrer les points dans le DataFrame 1 en fonction des coordonnées
    df_subset_centroid_min= df_centroid[(df_centroid['centroid-0'] >= dataframe2['xmin'][k]) & (df_centroid['centroid-0'] <= dataframe2['xmax'][k]) & (df_centroid['centroid-1'] >= dataframe2['ymin'][k]) & (df_centroid['centroid-1'] <= dataframe2['ymax'][k])]



    ####CENTROIDES PRESENTS DANS LA ZONE D'EXCLUSION

    df_subset_centroid_rejected = df_subset_centroid_max[~df_subset_centroid_max['label'].isin(df_subset_centroid_min['label'])]

    df_resultat= pd.concat([df_resultat,df_subset_centroid_rejected], ignore_index=True)









###ON SUPPRIME LES OBJETS SITUES DANS LA ZONE D4EXCLUSION
#resultat += df_centroid[~df_centroid['label'].isin(df_subset_centroid_rejected['label'])]


##print(df_subset_centroid_rejected)

#df_final = df_filtered[~df_filtered['label'].isin(df_subset_centroid_rejected['label'])]

# Affichage
#on_y_est = np.isin(cleared_label_image , df_final['label'])


# Show the filtered label_image
#plt.imshow(on_y_est)

In [ ]:
import pandas as pd

# Création des DataFrames exemple
data1 = {'Colonne1': ['A', 'B', 'C', 'D']}
data2 = {'Colonne1': ['B', 'D', 'E', 'F']}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Filtrage des lignes
df_filtered = df1MAX[~df1MAX['Colonne1'].isin(df2MIN['Colonne1'])]

# Affichage du DataFrame filtré
print(df_filtered)


In [ ]:
import pandas as pd

##### CENTROIDES PRESENTS DANS TRAY MAX

# Exemple de DataFrame 1
dataframe1 = df_centroid

# Exemple de DataFrame 2
dataframe2 = df_tray_max

# Récupérer les valeurs de la première ligne du DataFrame 2
valeur_min_x = dataframe2['xmin'][20]
valeur_max_x = dataframe2['xmax'][20]
valeur_min_y = dataframe2['ymin'][20]
valeur_max_y = dataframe2['ymax'][20]


# Filtrer les points dans le DataFrame 1 en fonction des coordonnées
df_subset_centroid_max= dataframe1[(dataframe1['centroid-0'] >= valeur_min_x) & (dataframe1['centroid-0'] <= valeur_max_x) & (dataframe1['centroid-1'] >= valeur_min_y) & (dataframe1['centroid-1'] <= valeur_max_y)]


#### CENTROIDES PRESENTS DANS LE TRAY MIN

# Exemple de DataFrame 1
dataframe1 = df_centroid

# Exemple de DataFrame 2
dataframe2 = df_tray_min

# Récupérer les valeurs de la première ligne du DataFrame 2
valeur_min_x = dataframe2['xmin'][20]
valeur_max_x = dataframe2['xmax'][20]
valeur_min_y = dataframe2['ymin'][20]
valeur_max_y = dataframe2['ymax'][20]


# Filtrer les points dans le DataFrame 1 en fonction des coordonnées
df_subset_centroid_min= dataframe1[(dataframe1['centroid-0'] >= valeur_min_x) & (dataframe1['centroid-0'] <= valeur_max_x) & (dataframe1['centroid-1'] >= valeur_min_y) & (dataframe1['centroid-1'] <= valeur_max_y)]


####CENTROIDES PRESENTS DANS LA ZONE D'EXCLUSION
df_subset_centroid_rejected= df_subset_centroid_max - df_subset_centroid_min

###ON SUPPRIME LES OBJETS SITUES DANS LA ZONE D4EXCLUSION
resultat = df_centroid[~df_centroid['label'].isin(df_subset_centroid_rejected['label'])]

print(df_subset_centroid_min)


In [ ]:
import numpy as np
import pandas as pd

x_mean_coords = []
y_mean_coords = []
labels = []
df_filtered_reinit=df_filtered.reset_index(drop=True)

for k in range(len(df_filtered_reinit)):
    data_coords = df_filtered_reinit['coords'][k]
    coords = data_coords.tolist()
    x_coords = coords[::2]
    y_coords = coords[1::2]

    mean_x = min(x_coords)
    mean_y = min(y_coords)

    x_mean_coords.append(mean_x)
    y_mean_coords.append(mean_y)
    labels.append(df_filtered_reinit['label'][k])

data_mean_pixel_coord = {'label': labels,
                         'x_mean': x_mean_coords,
                         'y_mean': y_mean_coords}

df_mean_pixel_coord = pd.DataFrame(data_mean_pixel_coord)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd

# Charger l'image
image = filtered_label_image

# Créer la figure et les axes
fig, ax = plt.subplots()

# Afficher l'image
ax.imshow(image)

centrioles_x = df_mean_pixel_coord ['x_mean'].tolist()
centrioles_y = df_mean_pixel_coord ['y_mean'].tolist()

# Tracer les centrioles
for x, y in zip(centrioles_x, centrioles_y):
    circle = patches.Circle((x, y), radius=10, color='red')
    ax.add_patch(circle)

# Afficher le tracé
plt.show()


# Pot segmentation

In [ ]:
from skimage.io import imread
import pandas as pd



# Charger l'image
image = imread('/home/mcaroulle/Pheno_Leaf/Data/Color segmentation/TRAY_1.png')

# Extraire les dimensions de l'image
height, width, channels = image.shape

#Calcul de la largeur et de la hauteur des pots dans le tray
x_pot = width/7
y_pot = height/5


# Création des données qui correspond à un découpage du tray en pots
x_min=[]
x_max=[]
X=[]

for k in range(5):
    x=0
    for k in range (8):
        X.append(int(x))
        x+=x_pot
    x_min+=X[0:7]
    x_max+=X[1:8]
    if x_max[-1]!=width:
        x_max[-1]=width


y_min=[]
y_max=[]
Y=[]


for k in range(7):
    y=0
    for k in range (6):
        Y.append(int(y))
        y+=y_pot
    y_min+=Y[0:5]
    y_max+=Y[1:6]
    if y_max[-1]!=height:
        y_max[-1]=height


data_tray_max = {'Pot_position': [i for i in range(1, 36)],
        'xmin': [k for k in x_min],
        'xmax': [k for k in x_max],
        'ymin': [k for k in y_min],
        'ymax': [k for k in y_max]}

df_tray_max = pd.DataFrame(data_tray_max)
#print(df_tray_max)



##création du tableau ne contenant que les centroïdes en filtrant le dataframe du masque obtenu précemment
df_centroid = df_filtered [['label','centroid-0','centroid-1']]

# on crée un df vide pour recueillir les résultats
data_resultat = {'label': [],
        'centroid-0': [],
        'centroid-1': []}

df_resultat = pd.DataFrame(data_resultat)


## Fusion des labels 

import pandas as pd


    

merged_label_image = np.copy(label_image)

for k in range (len(df_tray_max)):

    ##### CENTROIDES PRESENTS DANS TRAY MAX

    df_subset_centroid_max= df_centroid[(df_centroid['centroid-0'] >= df_tray_max['xmin'][k]) & (df_centroid['centroid-0'] <= df_tray_max['xmax'][k]) & (df_centroid['centroid-1'] >= df_tray_max['ymin'][k]) & (df_centroid['centroid-1'] <= df_tray_max['ymax'][k])]


    ### Fusion des labels des centroides présents
    #function wich fuse all the labels contained in a list

    list_labels =  df_subset_centroid_max['label'].tolist()
    
    for k in range (len(list_labels)-1):
        merged_label_image[merged_label_image == list_labels[k+1]] = list_labels[k]

    
    



image_label_overlay = label2rgb(merged_label_image, image=image)
plt.imshow(image_label_overlay)


###ON SUPPRIME LES OBJETS SITUES DANS LA ZONE D4EXCLUSION
#resultat += df_centroid[~df_centroid['label'].isin(df_subset_centroid_rejected['label'])]


##print(df_subset_centroid_rejected)

#df_final = df_filtered[~df_filtered['label'].isin(df_subset_centroid_rejected['label'])]

# Affichage
#on_y_est = np.isin(cleared_label_image , df_final['label'])


# Show the filtered label_image
#plt.imshow(on_y_est)

In [ ]:
from skimage import feature
from skimage import color
import cv2
import numpy as np
from scipy.ndimage import distance_transform_edt

###On définit une fonction qui permet de convertir un masque en une image
def convertir_masque_en_image(mask):
    # Créer une image vide avec les mêmes dimensions que le masque
    image = np.zeros_like(mask)

    # Appliquer le masque à l'image en définissant les pixels blancs
    image[mask > 0] = 255

    # Enregistrer l'image
    return image

image = convertir_masque_en_image(closed_mask)

edges = feature.canny(image, sigma=10)

dt = distance_transform_edt(~edges)

local_max = feature.peak_local_max(dt, min_distance=5)

plt.imshow(local_max, cmap='gray')
